# Sec Financial Statement Data Sets Tools - Quickstart

## TL;DR

This notebook gives a first introduction into using the secfsdstools (Sec Financial Data Sets Tools) python package: https://pypi.org/project/secfsdstools/

It is designed to work with the data provided by the "Sec Financial Statement Data Sets" (SFSDS)(https://www.sec.gov/dera/data/financial-statement-data-sets).

The SFSDS contains data from all reports that were filed with the sec since 2012. For instance all anual and quarter reports. The main asset that can be retrieved from this data set are the financial statemens (balance sheet, income statement, and cash flow).

First, this notebook shows how the library is installed and configured. After that, it shows how the financial statements can be extracted from the data set.

For a detailed definition of the data set see https://www.sec.gov/files/aqfs.pdf.

## Principles / Concepts

The libary will download all data files that are available. Currently, this is a total of 2GB and every year about 200MB are added. Every quarter has its own compressed file which contains the data from the quarter. But after, you have all the information on your disk and there is no need make additional calls to 

The reports are indexed in a simple sqlite database, which makes finding the a report much more efficient. As of the end of 2022, there are over 500'000 reports from more than 15'000 companies.

The library is storage and memory efficient. The data is directly read from the compressed files. Moreover, only the requested data is read and instantiated as pandas dataframes.

## Installation
In order to install the library, just use pip install:
```
pip install secfsdstools
```

## Configuration / Setup

In order to be used, the library needs to know where to store the compressed files from the SFSDS and where to store the sqlite database file. This is configured in a configuration file.

The easiest way to create the configuration file is to import the update method of the library and run it:

In [ ]:
from secfsdstools.update import update

update()

If you run the method for the first time, it will fail with the following message:
```
No config file found at home directory C:\Users\hansj\.secfsdstools.cfg.
Config file created at <user-home>/.secfsdstools.cfg. Please check the content and rerun.
```

As the message says, a default config file was created in your user home directory. It has the following content:
```
[DEFAULT]
downloaddirectory = <userhome>/secfsdstools/data/dld
dbdirectory = <userhome>/secfsdstools/data/db
useragentemail = your.email@goeshere.com
```
The downloaddirectory is the folder in which the compressed data files are downloaded and inside the fodler defined in dbdirectory the sqlite db file is created.
The useragentemail is set inside the header when requests to sec.gov are made. This should be your email-address, however, since we are only making very few requests, it doesn't really matter if you change it or not.

If you plan to use Jupyter, make sure that you configure the directories at a location where your Jupyter process has access. The used default directory (your user home directory) will work.

Next, run the update method again.

In [1]:
# to ensure that the logging statements are shown in juypter output
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
from secfsdstools.update import update

update()

INFO:root:    missing entries 24
INFO:secfsdstools.c_download.secdownloading:    start to download 2022q4.zip 
INFO:secfsdstools.c_download.secdownloading:    start to download 2022q3.zip 
INFO:secfsdstools.c_download.secdownloading:    start to download 2022q2.zip 
INFO:secfsdstools.c_download.secdownloading:    start to download 2022q1.zip 
INFO:secfsdstools.c_download.secdownloading:    start to download 2021q4.zip 
INFO:secfsdstools.c_download.secdownloading:    start to download 2021q3.zip 
INFO:secfsdstools.c_download.secdownloading:    start to download 2021q2.zip 
INFO:secfsdstools.c_download.secdownloading:    start to download 2021q1.zip 
INFO:secfsdstools.c_download.secdownloading:    start to download 2020q4.zip 
INFO:secfsdstools.c_download.secdownloading:    start to download 2020q3.zip 
INFO:secfsdstools.c_download.secdownloading:    start to download 2020q2.zip 
INFO:secfsdstools.c_download.secdownloading:    start to download 2020q1.zip 
INFO:secfsdstools.c_download.se

This may take a few minutes.

The new quarter zip files are available by the beginning of every quarter (January, April, July, October), hence, yo have to run the update() at the beginning of every quarter to get the data for the reprots from last quarter.

## Read Financial Statements for apple annual report 2022

### Finding the cik of the company

In order to read data for a company, we have to know its CIK number:

> Central Index Key (CIK). Ten digit number assigned by the SEC to each registrant that submits filings.
>
> <cite>https://www.sec.gov/files/aqfs.pdf</cite>

We can do this in several ways.
1. You can use the company search form on sec.gov: https://www.sec.gov/edgar/searchedgar/companysearch
1. You can use a sqlite db browsing tool (e.g., [DB Browser for SQLite](https://sqlitebrowser.org/)) DB and point it to the sqlite db file inside the directory you did configure in the config file above and search for the company in the index_reports table
1. use the `IndexSearch` class from the secfsdstools library as shown below


In [3]:
from secfsdstools.e_read.searching import IndexSearch

# initialize the search class
search = IndexSearch.get_index_search()

# search for the company name or part of it
result_df = search.find_company_by_name('apple')
result_df # show the result

,name,cik
0,"APPLE GREEN HOLDING, INC.",1510976
1,"APPLE HOSPITALITY REIT, INC.",1418121
2,APPLE INC,320193
3,"APPLE REIT EIGHT, INC.",1387361
4,"APPLE REIT NINE, INC.",1418121
5,"APPLE REIT SEVEN, INC.",1329011
6,APPLE REIT SIX INC,1277151
7,"APPLE REIT TEN, INC.",1498864
8,APPLETON PAPERS INC/WI,1144326
9,"DR PEPPER SNAPPLE GROUP, INC.",1418135


### Company information

"Apple Inc" seems to match our company the best, so lets check the details of the company.
In order to do that, we can use the `CompayReader`

In [3]:
from secfsdstools.e_read.companyreading import CompanyReader
apple_cik = 320193

apple_reader = CompanyReader.get_company_reader(cik=apple_cik)

Next, let us see what information was filed in the last report of that company:

In [4]:
apple_reader.get_latest_company_filing()

{'adsh': '0001193125-22-278435',
 'cik': '320193',
 'name': 'APPLE INC',
 'sic': '3571',
 'countryba': 'US',
 'stprba': 'CA',
 'cityba': 'CUPERTINO',
 'zipba': '95014',
 'bas1': 'ONE APPLE PARK WAY',
 'bas2': '',
 'baph': '(408) 996-1010',
 'countryma': 'US',
 'stprma': 'CA',
 'cityma': 'CUPERTINO',
 'zipma': '95014',
 'mas1': 'ONE APPLE PARK WAY',
 'mas2': '',
 'countryinc': 'US',
 'stprinc': 'CA',
 'ein': '942404110',
 'former': 'APPLE INC',
 'changed': '20070109',
 'afs': '1-LAF',
 'wksi': '0',
 'fye': '0930',
 'form': '8-K',
 'period': '20221031',
 'fy': '',
 'fp': '',
 'filed': '20221107',
 'accepted': '2022-11-07 06:27:00.0',
 'prevrpt': '0',
 'detail': '0',
 'instance': 'd400465d8k_htm.xml',
 'nciks': '1',
 'aciks': ''}

This are the information from the sub.txt file. To learn what the columns mean, see https://www.sec.gov/files/aqfs.pdf.

lets get a list of all the reports apple has filed in the last 12 years. There are two methods, one returns a list of `IndexReport` instances, the other ones a pandas dataframe with exactly the same information. To display the data, let's use the pandas DataFrame version:

In [5]:
apple_all_reports_df = apple_reader.get_all_company_reports_df()
apple_all_reports_df

,adsh,cik,name,form,filed,period,fullPath,originFile,originFileType,url
0,0001193125-22-278435,320193,APPLE INC,8-K,20221107,20221031.0,C:\Users\hansj\secfsdstools\data\dld\2022q4.zip,2022q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
1,0000320193-22-000107,320193,APPLE INC,8-K,20221027,20221031.0,C:\Users\hansj\secfsdstools\data\dld\2022q4.zip,2022q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
2,0000320193-22-000108,320193,APPLE INC,10-K,20221028,20220930.0,C:\Users\hansj\secfsdstools\data\dld\2022q4.zip,2022q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
3,0001193125-22-225365,320193,APPLE INC,8-K,20220819,20220831.0,C:\Users\hansj\secfsdstools\data\dld\2022q3.zip,2022q3.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
4,0001193125-22-214914,320193,APPLE INC,8-K,20220808,20220731.0,C:\Users\hansj\secfsdstools\data\dld\2022q3.zip,2022q3.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
...,...,...,...,...,...,...,...,...,...,...
82,0001193125-10-088957,320193,APPLE INC,10-Q,20100421,20100331.0,C:\Users\hansj\secfsdstools\data\dld\2010q2.zip,2010q2.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
83,0001193125-10-012085,320193,APPLE INC,10-Q,20100125,20091231.0,C:\Users\hansj\secfsdstools\data\dld\2010q1.zip,2010q1.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
84,0001193125-10-012091,320193,APPLE INC,10-K/A,20100125,20090930.0,C:\Users\hansj\secfsdstools\data\dld\2010q1.zip,2010q1.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
85,0001193125-09-214859,320193,APPLE INC,10-K,20091027,20090930.0,C:\Users\hansj\secfsdstools\data\dld\2009q4.zip,2009q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...


We are only interested in the annual reports (10-K), so lets filter for them

In [6]:
# of course, you could filter the apple_all_reports_df, but there is also a parameter in the get_all_company_reports_df methodapp
apple_10ks_df = apple_reader.get_all_company_reports_df(forms=['10-k'])
apple_10ks_df

,adsh,cik,name,form,filed,period,fullPath,originFile,originFileType,url
0,0000320193-22-000108,320193,APPLE INC,10-K,20221028,20220930.0,C:\Users\hansj\secfsdstools\data\dld\2022q4.zip,2022q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
1,0000320193-21-000105,320193,APPLE INC,10-K,20211029,20210930.0,C:\Users\hansj\secfsdstools\data\dld\2021q4.zip,2021q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
2,0000320193-20-000096,320193,APPLE INC,10-K,20201030,20200930.0,C:\Users\hansj\secfsdstools\data\dld\2020q4.zip,2020q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
3,0000320193-19-000119,320193,APPLE INC,10-K,20191031,20190930.0,C:\Users\hansj\secfsdstools\data\dld\2019q4.zip,2019q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
4,0000320193-18-000145,320193,APPLE INC,10-K,20181105,20180930.0,C:\Users\hansj\secfsdstools\data\dld\2018q4.zip,2018q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
5,0000320193-17-000070,320193,APPLE INC,10-K,20171103,20170930.0,C:\Users\hansj\secfsdstools\data\dld\2017q4.zip,2017q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
6,0001628280-16-020309,320193,APPLE INC,10-K,20161026,20160930.0,C:\Users\hansj\secfsdstools\data\dld\2016q4.zip,2016q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
7,0001193125-15-356351,320193,APPLE INC,10-K,20151028,20150930.0,C:\Users\hansj\secfsdstools\data\dld\2015q4.zip,2015q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
8,0001193125-14-383437,320193,APPLE INC,10-K,20141027,20140930.0,C:\Users\hansj\secfsdstools\data\dld\2014q4.zip,2014q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...
9,0001193125-13-416534,320193,APPLE INC,10-K,20131030,20130930.0,C:\Users\hansj\secfsdstools\data\dld\2013q4.zip,2013q4.zip,quarter,https://www.sec.gov/Archives/edgar/data/320193...


### Reading the details of a report

In order to get the details of a report, you either need the unique id of the report (the **adsh** identifier) or you need an `IndexReport` instance of that report.
First, let us see how this is can be done with the adsh identifier

In [7]:
from secfsdstools.e_read.reportreading import ReportReader

# getting the adsh of the latest 10-K report
apple_latest_10k_adsh = apple_10ks_df[:1].adsh.values[0]

apple_latest_10k_report_reader = ReportReader.get_report_by_adsh(adsh=apple_latest_10k_adsh)

The same is possible with an instance of `IndexReport` class:

In [8]:
# this returns a list of IndexReport instances, instead of a pandas DataFrame. They are sorted by date, so the first entry is the latest report
apple_latest_10k_indexreport = apple_reader.get_all_company_reports(forms=['10-k'])[0]
apple_latest_10k_report_reader = ReportReader.get_report_by_indexreport(index_report=apple_latest_10k_indexreport)

Next, let us have a look at the balance sheet. First we get the data for the financial statements. We make sure that all information from the period of the report and the previoius period are contained. Therefore, we use the `financial_statements_for_period_and_previous_period()` method:

In [9]:
apple_latest_10k_fs_df = apple_latest_10k_report_reader.financial_statements_for_period_and_previous_period()
apple_latest_10k_fs_df[apple_latest_10k_fs_df.stmt == 'BS']

,adsh,tag,version,stmt,report,line,uom,negating,inpth,20210930,20220930,form
0,0000320193-22-000108,CashAndCashEquivalentsAtCarryingValue,us-gaap/2022,BS,5,3,USD,0,0,3.494000e+10,2.364600e+10,10-K
1,0000320193-22-000108,MarketableSecuritiesCurrent,us-gaap/2022,BS,5,4,USD,0,0,2.769900e+10,2.465800e+10,10-K
2,0000320193-22-000108,AccountsReceivableNetCurrent,us-gaap/2022,BS,5,5,USD,0,0,2.627800e+10,2.818400e+10,10-K
3,0000320193-22-000108,InventoryNet,us-gaap/2022,BS,5,6,USD,0,0,6.580000e+09,4.946000e+09,10-K
4,0000320193-22-000108,NontradeReceivablesCurrent,us-gaap/2022,BS,5,7,USD,0,0,2.522800e+10,3.274800e+10,10-K
5,0000320193-22-000108,OtherAssetsCurrent,us-gaap/2022,BS,5,8,USD,0,0,1.411100e+10,2.122300e+10,10-K
6,0000320193-22-000108,AssetsCurrent,us-gaap/2022,BS,5,9,USD,0,0,1.348360e+11,1.354050e+11,10-K
7,0000320193-22-000108,MarketableSecuritiesNoncurrent,us-gaap/2022,BS,5,11,USD,0,0,1.278770e+11,1.208050e+11,10-K
8,0000320193-22-000108,PropertyPlantAndEquipmentNet,us-gaap/2022,BS,5,12,USD,0,0,3.944000e+10,4.211700e+10,10-K
9,0000320193-22-000108,OtherAssetsNoncurrent,us-gaap/2022,BS,5,13,USD,0,0,4.884900e+10,5.442800e+10,10-K


In the same way, you can get the data for the Income Statement (IS) or the Cash Flow (CF).

Did you notice that the report data also includes the url where you can view the filed report online?

In [10]:
apple_latest_10k_indexreport.url

'https://www.sec.gov/Archives/edgar/data/320193/000032019322000108/0000320193-22-000108-index.htm'

If you open the link and click on the first entry in the table (the file: aapl-20220924.htm), it will open the real filed report: https://www.sec.gov/ix?doc=/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm

If you open it and click on the "sections" in the top menu bar, you can directly navigate to the financial statements in the report. Of course, you can also scroll to them.

Open then balance sheet and see for yourself, that the data above matches the shown values and are in the same order.

## Multireport reading

The ReportReader reads the data for a single report, but there are also other readers who can read multiple reports at once, but share the same interface as the ReportReader.

### ZipReportReader

The ZipReportReader reads all reports of a single zip file at once.

In [2]:
from secfsdstools.e_read.zipreportreading import ZipReportReader

reader = ZipReportReader.get_zip_by_name(name='2020q1.zip')

In [3]:
# show the submissions entries
reader.get_raw_sub_data()

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,baph,countryma,stprma,cityma,zipma,mas1,mas2,countryinc,stprinc,ein,former,changed,afs,wksi,fye,form,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks
0,0000004977-20-000002,4977,AFLAC INC,6321.0,US,GA,COLUMBUS,31999,1932 WYNNTON RD,NaN,7063233431,US,GA,COLUMBUS,31999,1932 WYNNTON ROAD,NaN,US,GA,581167100,AMERICAN FAMILY CORP,19920306.0,1-LAF,0,1231.0,8-K,20191231,NaN,NaN,20200102,2020-01-02 07:45:00.0,0,0,afl8-kxdecember2019xot_htm.xml,1,NaN
1,0000018498-20-000002,18498,GENESCO INC,5661.0,US,TN,NASHVILLE,37217,GENESCO PK 1415 MURFREESBORO RD,NaN,6153677000,US,TN,NASHVILLE,37217,GENESCO PK 1415 MURFREESBORO RD,NaN,US,TN,620211340,NaN,NaN,1-LAF,0,131.0,8-K,20191231,NaN,NaN,20200102,2020-01-02 17:05:00.0,0,0,a8-ktogast010220_htm.xml,1,NaN
2,0000052795-20-000004,52795,ANIXTER INTERNATIONAL INC,5063.0,US,IL,GLENVIEW,60026,2301 PATRIOT BLVD,NaN,2245218204,US,IL,GLENVIEW,60026,2301 PATRIOT BLVD,NaN,US,DE,941658138,ITEL CORP,19920703.0,1-LAF,0,1231.0,8-K,20191231,NaN,NaN,20200102,2020-01-02 06:38:00.0,1,0,a8-kxsecondamendedmerg_htm.xml,1,NaN
3,0000082811-20-000002,82811,REGAL BELOIT CORP,3621.0,US,WI,BELOIT,53511,200 STATE ST,NaN,6083648800,US,WI,BELOIT,53511-6254,200 STATE STREET,NaN,US,WI,390875718,BELOIT TOOL CORP,19730522.0,1-LAF,0,1231.0,8-K,20191231,NaN,NaN,20200102,2020-01-02 09:09:00.0,0,0,a8-kdec272019cover_htm.xml,1,NaN
4,0000091767-20-000003,91767,SONOCO PRODUCTS CO,2650.0,US,SC,HARTSVILLE,29551-0160,ONE NORTH SECOND ST,P O BOX 160,8433837000,US,SC,HARTSVILLE,29550,ONE N. SECOND STREET,NaN,US,SC,570248420,NaN,NaN,1-LAF,0,1231.0,8-K,20191231,NaN,NaN,20200102,2020-01-02 11:55:00.0,1,0,son-20191231_htm.xml,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13556,0001664703-20-000013,1664703,BLOOM ENERGY CORP,3620.0,US,CA,SAN JOSE,95134,4353 NORTH FIRST STREET,NaN,408-543-1500,US,CA,SAN JOSE,95134,4353 NORTH FIRST STREET,NaN,US,DE,770565408,NaN,NaN,2-ACC,0,1231.0,10-K,20191231,2019.0,FY,20200331,2020-03-31 17:22:00.0,0,1,be-20191231.xml,1,NaN
13557,0001673985-20-000053,1673985,ADVANSIX INC.,2821.0,US,NJ,PARSIPPANY,07054,300 KIMBALL DRIVE,SUITE 101,(973) 526-1800,US,NJ,PARSIPPANY,07054,300 KIMBALL DRIVE,SUITE 101,US,DE,812525089,NaN,NaN,1-LAF,0,1231.0,8-K,20200331,NaN,NaN,20200331,2020-03-31 06:32:00.0,0,0,asix-20200331_htm.xml,1,NaN
13558,0001694426-20-000040,1694426,"DELEK US HOLDINGS, INC.",2911.0,US,TN,BRENTWOOD,37027,7102 COMMERCE WAY,NaN,615-721-3715,US,TN,BRENTWOOD,37027,7102 COMMERCE WAY,NaN,US,DE,352581557,"DELEK HOLDCO, INC.",20170111.0,1-LAF,0,1231.0,8-K,20200331,NaN,NaN,20200331,2020-03-31 17:28:00.0,0,0,dk-8kxannouncementofdr_htm.xml,1,NaN
13559,0001721868-20-000167,945617,"AMERICAN CANNABIS COMPANY, INC.",4833.0,US,CO,DENVER,80216,5690 LOGAN ST,SUITE A,303-974-4770,US,CO,DENVER,80216,5690 LOGAN ST,SUITE A,US,DE,942901715,"BRAZIL INTERACTIVE MEDIA, INC.",20130617.0,4-NON,0,1231.0,10-K,20191231,2019.0,FY,20200331,2020-03-31 12:25:00.0,0,1,ammj-20191231.xml,1,NaN


Since the ZipReportReader shares the interface of the ReportReader, you can, for instance, show the values for the "period" date of all reports at once

In [4]:
reader.financial_statements_for_period()

,adsh,coreg,tag,version,stmt,report,line,uom,negating,inpth,20111231,20120331,20120630,20120930,20121231,20130331,20130630,20130930,20131231,20140331,20140630,20140930,20141231,20150331,20150630,20150930,20151231,20160331,20160630,20160930,20161231,20170331,20170630,20170930,20171231,20180131,20180331,20180430,20180630,20180731,20180831,20180930,20181031,20181130,20181231,20190131,20190228,20190331,20190430,20190531,20190630,20190731,20190831,20190930,20191031,20191130,20191231,20200131,20200229,form
0,0000002178-20-000013,-,CashAndCashEquivalentsAtCarryingValue,us-gaap/2019,BS,2,3,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.129940e+08,NaN,NaN,10-K
1,0000002178-20-000013,-,RestrictedCashCurrent,us-gaap/2019,BS,2,4,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.261000e+06,NaN,NaN,10-K
2,0000002178-20-000013,-,AccountsReceivableNetCurrent,us-gaap/2019,BS,2,5,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.453400e+07,NaN,NaN,10-K
3,0000002178-20-000013,-,AccountsReceivableRelatedPartiesCurrent,us-gaap/2019,BS,2,6,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,10-K
4,0000002178-20-000013,-,EnergyRelatedInventory,us-gaap/2019,BS,2,7,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.640700e+07,NaN,NaN,10-K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621738,0001793498-20-000014,-,IncomeLossFromContinuingOperationsPerDilutedShare,us-gaap/2019,IS,2,31,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.000000e-02,NaN,NaN,10-K
621739,0001793498-20-000014,-,IncomeLossFromDiscontinuedOperationsNetOfTaxPe...,us-gaap/2019,IS,2,32,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,10-K
621740,0001793498-20-000014,-,EarningsPerShareDiluted,us-gaap/2019,IS,2,33,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.000000e-02,NaN,NaN,10-K
621741,0001793498-20-000014,-,WeightedAverageNumberOfSharesOutstandingBasic,us-gaap/2019,IS,2,34,shares,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.230000e+08,NaN,NaN,10-K


### CompanyCollector
The Company Collector can read all reports of a company at once. This means it collects all the information from all zip files. This is done in parallel.

In [5]:
from secfsdstools.e_read.companycollecting import CompanyReportCollector

# read all 10-K reports of apple
reader = CompanyReportCollector.get_company_collector(cik=320193, forms=['10-K'])

In [6]:
# show the submissions entries -> note: it will take some time, till the data from all the different zipfiles are loaded
reader.get_raw_sub_data()

,index,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,baph,countryma,stprma,cityma,zipma,mas1,mas2,countryinc,stprinc,ein,former,changed,afs,wksi,fye,form,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks
0,0,0000320193-22-000108,320193,APPLE INC,3571,US,CA,CUPERTINO,95014,ONE APPLE PARK WAY,NaN,(408) 996-1010,US,CA,CUPERTINO,95014,ONE APPLE PARK WAY,NaN,US,CA,942404110,APPLE INC,20070109,1-LAF,1,930,10-K,20220930,2022,FY,20221028,2022-10-27 18:01:00.0,0,1,aapl-20220924_htm.xml,1,NaN
1,0,0000320193-21-000105,320193,APPLE INC,3571,US,CA,CUPERTINO,95014,ONE APPLE PARK WAY,NaN,(408) 996-1010,US,CA,CUPERTINO,95014,ONE APPLE PARK WAY,NaN,US,CA,942404110,APPLE INC,20070109,1-LAF,1,930,10-K,20210930,2021,FY,20211029,2021-10-28 18:04:00.0,0,1,aapl-20210925_htm.xml,1,NaN
2,0,0000320193-20-000096,320193,APPLE INC,3571,US,CA,CUPERTINO,95014,ONE APPLE PARK WAY,NaN,(408) 996-1010,US,CA,CUPERTINO,95014,ONE APPLE PARK WAY,NaN,US,CA,942404110,APPLE INC,20070109,1-LAF,1,930,10-K,20200930,2020,FY,20201030,2020-10-29 18:06:00.0,0,1,aapl-20200926_htm.xml,1,NaN
3,0,0000320193-19-000119,320193,APPLE INC,3571,US,CA,CUPERTINO,95014,ONE APPLE PARK WAY,NaN,(408) 996-1010,US,CA,CUPERTINO,95014,ONE APPLE PARK WAY,NaN,US,CA,942404110,APPLE INC,20070109,1-LAF,1,930,10-K,20190930,2019,FY,20191031,2019-10-30 18:13:00.0,0,1,a10-k20199282019_htm.xml,1,NaN
4,0,0000320193-18-000145,320193,APPLE INC,3571,US,CA,CUPERTINO,95014,ONE APPLE PARK WAY,NaN,(408) 996-1010,US,CA,CUPERTINO,95014,ONE APPLE PARK WAY,NaN,US,CA,942404110,APPLE COMPUTER INC,19970808,1-LAF,1,930,10-K,20180930,2018,FY,20181105,2018-11-05 08:02:00.0,0,1,aapl-20180929.xml,1,NaN
5,0,0000320193-17-000070,320193,APPLE INC,3571,US,CA,CUPERTINO,95014,ONE INFINITE LOOP,NaN,(408) 996-1010,US,CA,CUPERTINO,95014,ONE INFINITE LOOP,NaN,US,CA,942404110,APPLE COMPUTER INC,19970808,1-LAF,1,930,10-K,20170930,2017,FY,20171103,2017-11-03 08:02:00.0,0,1,aapl-20170930.xml,1,NaN
6,0,0001628280-16-020309,320193,APPLE INC,3571,US,CA,CUPERTINO,95014,ONE INFINITE LOOP,NaN,(408) 996-1010,US,CA,CUPERTINO,95014,ONE INFINITE LOOP,NaN,US,CA,942404110,APPLE COMPUTER INC,19970808,1-LAF,1,930,10-K,20160930,2016,FY,20161026,2016-10-26 16:42:00.0,0,1,aapl-20160924.xml,1,NaN
7,0,0001193125-15-356351,320193,APPLE INC,3571,US,CA,CUPERTINO,95014,ONE INFINITE LOOP,NaN,(408) 996-1010,US,CA,CUPERTINO,95014,ONE INFINITE LOOP,NaN,US,CA,942404110,APPLE COMPUTER INC,19970808,1-LAF,1,930,10-K,20150930,2015,FY,20151028,2015-10-28 16:31:00.0,0,1,aapl-20150926.xml,1,NaN
8,0,0001193125-14-383437,320193,APPLE INC,3571,US,CA,CUPERTINO,95014,ONE INFINITE LOOP,NaN,(408) 996-1010,US,CA,CUPERTINO,95014,ONE INFINITE LOOP,NaN,US,CA,942404110,APPLE COMPUTER INC,19970808,1-LAF,1,930,10-K,20140930,2014,FY,20141027,2014-10-27 17:12:00.0,0,1,aapl-20140927.xml,1,NaN
9,0,0001193125-13-416534,320193,APPLE INC,3571,US,CA,CUPERTINO,95014,ONE INFINITE LOOP,NaN,(408) 996-1010,US,CA,CUPERTINO,95014,ONE INFINITE LOOP,NaN,US,CA,942404110,APPLE COMPUTER INC,19970808,1-LAF,1,930,10-K,20130930,2013,FY,20131030,2013-10-29 20:38:00.0,0,1,aapl-20130928.xml,1,NaN


In [7]:
# also hear, lets have a look at the returned dataframe of financial_statements_for_period()
reader.financial_statements_for_period()

,adsh,coreg,tag,version,stmt,report,line,uom,negating,inpth,20090930,20100930,20110930,20120930,20130930,20140930,20150930,20160930,20170930,20180930,20190930,20200930,20210930,20220930,form
0,0000320193-17-000070,-,CashAndCashEquivalentsAtCarryingValue,us-gaap/2017,BS,5,3,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.028900e+10,NaN,NaN,NaN,NaN,NaN,10-K
1,0000320193-17-000070,-,AvailableForSaleSecuritiesCurrent,us-gaap/2017,BS,5,4,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.389200e+10,NaN,NaN,NaN,NaN,NaN,10-K
2,0000320193-17-000070,-,AccountsReceivableNetCurrent,us-gaap/2017,BS,5,5,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.787400e+10,NaN,NaN,NaN,NaN,NaN,10-K
3,0000320193-17-000070,-,InventoryNet,us-gaap/2017,BS,5,6,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.855000e+09,NaN,NaN,NaN,NaN,NaN,10-K
4,0000320193-17-000070,-,NontradeReceivablesCurrent,us-gaap/2017,BS,5,7,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.779900e+10,NaN,NaN,NaN,NaN,NaN,10-K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,0001628280-16-020309,-,EarningsPerShareBasic,us-gaap/2015,IS,2,14,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.015000e+00,NaN,NaN,NaN,NaN,NaN,NaN,10-K
1391,0001628280-16-020309,-,EarningsPerShareDiluted,us-gaap/2015,IS,2,15,USD,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.990000e+00,NaN,NaN,NaN,NaN,NaN,NaN,10-K
1392,0001628280-16-020309,-,WeightedAverageNumberOfSharesOutstandingBasic,us-gaap/2015,IS,2,17,shares,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.470820e+09,NaN,NaN,NaN,NaN,NaN,NaN,10-K
1393,0001628280-16-020309,-,WeightedAverageNumberOfDilutedSharesOutstanding,us-gaap/2015,IS,2,18,shares,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.500281e+09,NaN,NaN,NaN,NaN,NaN,NaN,10-K


In [9]:
# We can also get the merged (pre and num data merged) but not pivoted data by using the merge method.
# There are also options to directly filter the data. So we want only see the Assets tag and for every report,
# we want only see the value where ddate equals the period date (use_period=True)
asset_values_df = reader.merge_pre_and_num(use_period=True, tags=['Assets'])

In [10]:
asset_values_df.sort_values(['ddate'], ascending=True)

,index_x,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote,index_y,report,line,stmt,inpth,rfile,plabel,negating
13,17,0001193125-09-214859,Assets,us-gaap/2009,-,20090930,0,USD,5.385100e+10,NaN,10,1,17,BS,0,X,Total assets,0
12,35,0001193125-10-238044,Assets,us-gaap/2009,-,20100930,0,USD,7.518300e+10,NaN,10,3,18,BS,0,X,Total assets,0
11,31,0001193125-11-282113,Assets,us-gaap/2011,-,20110930,0,USD,1.163710e+11,NaN,9,3,18,BS,0,H,Total assets,0
10,33,0001193125-12-444068,Assets,us-gaap/2012,-,20120930,0,USD,1.760640e+11,NaN,9,3,18,BS,0,H,Total assets,0
9,3,0001193125-13-416534,Assets,us-gaap/2013,-,20130930,0,USD,2.070000e+11,NaN,3,5,18,BS,0,H,Total assets,0
8,11,0001193125-14-383437,Assets,us-gaap/2014,-,20140930,0,USD,2.318390e+11,NaN,5,5,18,BS,0,H,Total assets,0
7,11,0001193125-15-356351,Assets,us-gaap/2015,-,20150930,0,USD,2.904790e+11,NaN,5,5,18,BS,0,H,Total assets,0
6,11,0001628280-16-020309,Assets,us-gaap/2015,-,20160930,0,USD,3.216860e+11,NaN,5,5,15,BS,0,H,Total assets,0
5,24,0000320193-17-000070,Assets,us-gaap/2017,-,20170930,0,USD,3.753190e+11,NaN,23,5,15,BS,0,H,Total assets,0
4,125,0000320193-18-000145,Assets,us-gaap/2018,-,20180930,0,USD,3.657250e+11,NaN,30,5,15,BS,0,H,Total assets,0
